In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
from policy_agent import VECTOR_SEARCH_TOOLS
import mlflow
from mlflow.models.resources import DatabricksFunction
from pkg_resources import get_distribution

resources = []
for tool in VECTOR_SEARCH_TOOLS:
    resources.extend(tool.resources)

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="foodly-ai-assistant",
        code_paths=["helpers.py"],
        python_model="policy_agent.py",
        pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}",
            f"mlflow=={get_distribution('mlflow').version}",
            f"databricks-langchain=={get_distribution('databricks-langchain').version}",
            f"langgraph=={get_distribution('langgraph').version}",
        ],
        resources=resources,
    )

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "agents"
schema = "main"
model_name = "foodly-ai-assistant"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/foodly-ai-assistant",
    input_data={"messages": [{"role": "user", "content": "What are foodly return policies?"}]},
    env_manager="uv",
)

In [0]:
from databricks import agents

agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    scale_to_zero=True,
    tags={"name": "foodly-ai-assistant-endpoint","creator" : "anirvan-sen"},
)